# Problem Statement

The Bach chorales dataset is made of 382 chorales composed by Johann Sebastian Bach. Each chorale is 100 to 640 time steps long, and each time step contains 4 integers, where each integer corresponds to a note's index on a piano.

We are trying to train a model that can predict the next four notes from a sequene of time steps.

# Import Libraries

In [7]:
from tensorflow import keras
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import SimpleRNN
import pandas as pd
import numpy as np
import os

# Load Data

In [2]:
def readfiles(path):
    X = []
    for filename in os.listdir(path):
        file_path = os.path.join(path, filename)
        df = pd.read_csv(file_path)
        X.append(df.values.tolist())
    return X

In [3]:
# Load Data
train = readfiles("./jsb_chorales/train")
test = readfiles("./jsb_chorales/test")
valid = readfiles("./jsb_chorales/valid")


In [4]:
print(len(train), 'train sequences')
print(len(valid), 'valid sequences')
print(len(test), 'test sequences')

229 train sequences
77 test sequences
76 valid sequences


In [8]:
train_pad = pad_sequences(train, dtype=np.float32)
valid_pad = pad_sequences(valid, dtype=np.float32)
test_pad = pad_sequences(test, dtype=np.float32)

# Build a RNN

## SimpleRNN

In [11]:
model_rnn = Sequential([
    SimpleRNN(20, return_sequences=True),
    SimpleRNN(20, return_sequences=True),
    SimpleRNN(1)
])